In [ ]:
from skimage import io
from skimage.transform import resize
import skimage
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from ast import literal_eval
from skimage.util import crop
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
import math
from scipy import ndimage
import traceback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img = pd.read_csv('/content/drive/MyDrive/img.csv')

In [ ]:
img = img.rename(columns={'id': 'image_id'}).copy()

In [ ]:
img = img.drop(columns=['Unnamed: 0']).copy()

In [ ]:
annot = pd.read_csv('/content/drive/MyDrive/annot.csv')

In [ ]:
annot = annot.drop(columns=['Unnamed: 0']).copy()

In [ ]:
annot.head()

,id,image_id,bbox,utf8_string,points,area
0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90
1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,...",3483.69
2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6...",3071.27
3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9...",6341.30
4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,...",2350.69


In [ ]:
img.head()

,image_id,width,height,set,file_name
0,a4ea732cd3d5948a,840,1024,train,train/a4ea732cd3d5948a.jpg
1,4bf43a7b2a898044,1024,683,train,train/4bf43a7b2a898044.jpg
2,1b55b309b0f50d02,1024,683,train,train/1b55b309b0f50d02.jpg
3,00c359f294f7dcd9,1024,680,train,train/00c359f294f7dcd9.jpg
4,04b5a37f762b0f51,768,1024,train,train/04b5a37f762b0f51.jpg


In [ ]:
cols = ['image_id']
annot = annot.join(img.set_index(cols), on=cols).copy()
#turns out not needed since all data are labeled as training set, but still useful to have the colunm for now

In [ ]:
annot.head()

,id,image_id,bbox,utf8_string,points,area,width,height,set,file_name
0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,"[525.83, 3.4, 197.64, 33.94]",Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525...",6707.90,840,1024,train,train/a4ea732cd3d5948a.jpg
1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,"[534.67, 64.68, 91.22, 38.19]",Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,...",3483.69,840,1024,train,train/a4ea732cd3d5948a.jpg
2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,"[626.95, 63.62, 96.52, 31.82]",Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6...",3071.27,840,1024,train,train/a4ea732cd3d5948a.jpg
3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,"[577.4, 141.87, 147.13, 43.1]",...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9...",6341.30,840,1024,train,train/a4ea732cd3d5948a.jpg
4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,"[391.03, 163.9, 60.82, 38.65]",.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,...",2350.69,840,1024,train,train/a4ea732cd3d5948a.jpg


In [ ]:
mean_w = annot['width'].mean()
std_w = annot['width'].std()
max_w = annot['width'].max()
min_w = annot['width'].min()

std_h = annot['height'].std()
mean_h = annot['height'].mean()
max_h = annot['height'].max()
min_h = annot['height'].min()

print('Mean Image Width: {},\n\tSTD Image Width: {},\n\tMax Image Width: {},\n\tMin Image Width: {},\nMean Image Height: {},\n\tSTD Image Height: {},\n\tMax Image Height: {},\n\tMin Image Height: {}'.format(mean_w, std_w, max_w, min_w, mean_h, std_w, max_h, min_h))

Mean Image Width: 956.0872463068512,
	STD Image Width: 139.52693910372935,
	Max Image Width: 6000,
	Min Image Width: 259,
Mean Image Height: 811.58137375826,
	STD Image Height: 139.52693910372935,
	Max Image Height: 4608,
	Min Image Height: 257


In [ ]:
annot = annot.drop(columns=['bbox', 'area', 'width', 'height', 'set', 'file_name']).copy()

In [ ]:
annot.columns

Index(['id', 'image_id', 'utf8_string', 'points'], dtype='object')

In [ ]:
annot

,id,image_id,utf8_string,points
0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525..."
1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,..."
2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6..."
3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9..."
4,a4ea732cd3d5948a_5,a4ea732cd3d5948a,.,"[395.2, 163.9, 451.85, 191.94, 445.59, 202.55,..."
...,...,...,...,...
1052349,0ebbecdc46b78d42_15,0ebbecdc46b78d42,.,"[295.65, -0.14, 295.65, 27.33, 267.47, 27.03, ..."
1052350,a37e1fb026b80a6d_1,a37e1fb026b80a6d,RÖR,"[331.69, 466.97, 749.0, 462.84, 749.0, 659.79,..."
1052351,a37e1fb026b80a6d_2,a37e1fb026b80a6d,Moderna,"[876.75, 287.61, 912.34, 285.63, 913.73, 293.9..."
1052352,a37e1fb026b80a6d_3,a37e1fb026b80a6d,Museet,"[913.53, 284.84, 944.77, 282.86, 945.76, 292.3..."


In [ ]:
! mkdir drive/MyDrive/fixed_word_image_set

In [ ]:
#! mkdir testing_images_saved

In [ ]:
#second version
def cropRectangle(points, image):
    angle = -math.tan((points[3] - points[1])/(points[2] - points[0]))/2 - math.tan((points[7] - points[5])/(points[6] - points[4]))/2
    rotated = ndimage.rotate(image, angle * 180 / math.pi)
    point1x = (points[0] - image.shape[0]/2) * math.cos(angle) - (points[1] - image.shape[1]/2) * math.sin(angle) + rotated.shape[0]/2
    point1y = (points[1] - image.shape[1]/2) * math.cos(angle) + (points[0] - image.shape[0]/2) * math.sin(angle) + rotated.shape[1]/2
    point2x = (points[4] - image.shape[0]/2) * math.cos(angle) - (points[5] - image.shape[1]/2) * math.sin(angle) + rotated.shape[0]/2
    point2y = (points[5] - image.shape[1]/2) * math.cos(angle) + (points[4] - image.shape[0]/2) * math.sin(angle) + rotated.shape[1]/2
    #print(point1x, point1y, point2x, point2y)
    final = rotated[int(min(point1y, point2y)):int(max(point1y, point2y)), int(min(point1x, point2x)):int(max(point1x, point2x))]
    if final.shape[0] > final.shape[1]:
        final = ndimage.rotate(final, 90)
    final = cv2.cvtColor(final, cv2.COLOR_BGR2GRAY)#grayscaling
    return final

In [ ]:
dataset_path = '/content/drive/MyDrive/train_images'
pattern = re.compile("^[^\w\s]+$")#strings that are just puncations
words_in_img = None

i = 2000
while i < 3000:
  print(i)#if reaches error just skip image and continue next
  try:
    #if i in [128, 166, 406]:
      #i+=1
      #continue
    image_id = img.iloc[i]['image_id']
    image_name = image_id + '.jpg'
    image_path = os.path.join(dataset_path, image_name)
    #image = io.imread(image_path)
    image = cv2.imread(image_path)
    gray_ed = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#grayscaling
    norm_ed = (gray_ed - np.min(gray_ed)) / (np.max(gray_ed) - np.min(gray_ed))#normalizing

    words_temp = annot[annot['image_id'] == image_id].copy()
    words_temp = words_temp[words_temp['utf8_string'].str.match(pattern) == False].copy()#check and drop strings that are just punctuations
    if words_temp.empty:#make sure after removing the df is not empty
      i += 1
      continue

    words_temp_points = list(words_temp['points'])
    words_temp_id = list(words_temp['id'])

    id_count = 0
    temp_img_list = []
    set_len = len(words_temp_points)
    while id_count < set_len:
    #for x in words_temp_points:
      crop_coord = np.array(literal_eval(words_temp_points[id_count]))#to crop out each individual word
      cropped = cropRectangle(crop_coord, image)
      # convert each image of shape (32, 128, 1) without streching/distorting too much by scaling and padding
      h,w = cropped.shape
      if w < 10 or h < 10:
        id_count += 1
        continue
      cropped = cv2.resize(cropped, (int(w * 32 / h), 32), interpolation= cv2.INTER_LINEAR)
      if w >= 128:
        cropped = cv2.resize(cropped, (128, 32))
      else:
        w_pad = 128 - int(w * 32 / h)
        cropped = cv2.copyMakeBorder(cropped, 0, 0, 0, w_pad, cv2.BORDER_CONSTANT, None, 0)

      temp_img_list.append(cropped)
      id_count += 1
  except Exception:
    i += 1
    traceback.print_exc()
    continue

  id_count = 0
  for y in temp_img_list:
    #plt.imsave('testing_images_saved/' + words_temp_id[id_count] + '.jpg', y, cmap='gray')
    plt.imsave('drive/MyDrive/fixed_word_image_set/' + words_temp_id[id_count] + '.jpg', y, cmap='gray')
    id_count += 1

  if words_in_img is None:
    words_in_img = words_temp.copy()
  else:
    words_in_img = pd.concat([words_in_img, words_temp])

  words_in_img.to_csv('drive/MyDrive/words_in_img.csv')
  i += 1

In [ ]:
'''
import shutil

shutil.rmtree('testing_images_saved')
'''

In [ ]:
'''
import shutil

shutil.rmtree('/content/drive/MyDrive/processed_word_images')
'''

In [ ]:
words_in_img

,id,image_id,utf8_string,points
0,a4ea732cd3d5948a_1,a4ea732cd3d5948a,Performance,"[525.83, 3.4, 723.47, 7.29, 722.76, 36.99, 525..."
1,a4ea732cd3d5948a_2,a4ea732cd3d5948a,Sport,"[535.73, 64.68, 623.41, 67.51, 625.89, 102.87,..."
2,a4ea732cd3d5948a_3,a4ea732cd3d5948a,Watch,"[626.95, 63.62, 721.7, 63.62, 723.47, 95.44, 6..."
3,a4ea732cd3d5948a_4,a4ea732cd3d5948a,...period.,"[580.02, 143.61, 724.53, 141.87, 723.66, 184.9..."
5,a4ea732cd3d5948a_6,a4ea732cd3d5948a,400,"[464.54, 204.45, 477.61, 221.88, 469.85, 229.2..."
...,...,...,...,...
15166,a19634159fe9430e_7,a19634159fe9430e,pe.eu,"[406.66, 489.33, 430.79, 430.24, 454.21, 435.2..."
15168,a19634159fe9430e_9,a19634159fe9430e,NOH,"[579.54, 668.11, 635.74, 668.47, 636.28, 680.1..."
15170,a19634159fe9430e_11,a19634159fe9430e,NOH,"[13.73, 663.47, 40.39, 663.29, 39.85, 673.38, ..."
15173,33b009b5614ce4d3_1,33b009b5614ce4d3,uernonchan.com,"[752.13, 615.62, 1006.02, 616.61, 1007.0, 650...."


In [ ]:
import string
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#image_id = img.iloc[i]['image_id']
#image_name = image_id + '.jpg'
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images
#image_path = os.path.join(dataset_path, image_name)
#image = cv2.imread(image_path)

In [ ]:
df1 = pd.read_csv('drive/MyDrive/k_csvs/0-99.csv')
df2 = pd.read_csv('drive/MyDrive/k_csvs/100-559.csv')
df3 = pd.read_csv('drive/MyDrive/k_csvs/560-749.csv')

In [ ]:
final_image_csvs = pd.concat([df1, df2])
final_image_csvs = pd.concat([final_image_csvs, df3]).reset_index()
final_image_csvs = final_image_csvs.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'index']).copy()

In [ ]:
final_image_csvs

,id,image_id,utf8_string,points
0,00c359f294f7dcd9_1,00c359f294f7dcd9,G-ATCO,"[322.37, 391.39, 436.35, 368.64, 441.0, 400.44..."
1,00c359f294f7dcd9_2,00c359f294f7dcd9,HUSKY,"[865.36, 327.84, 897.5, 322.28, 899.28, 328.4,..."
2,04b5a37f762b0f51_1,04b5a37f762b0f51,OUR,"[218.66, 302.27, 316.51, 303.52, 315.68, 347.6..."
3,04b5a37f762b0f51_2,04b5a37f762b0f51,"NEIGHBORS,","[329.21, 301.01, 615.75, 301.59, 616.9, 355.1,..."
4,04b5a37f762b0f51_3,04b5a37f762b0f51,THE,"[248.65, 362.58, 353.37, 360.85, 353.37, 407.4..."
...,...,...,...,...
3653,4cefcb45a504cee4_1,4cefcb45a504cee4,SENIOR,"[763.83, 272.8, 886.15, 284.98, 884.2, 324.45,..."
3654,c21528d4223161d8_1,c21528d4223161d8,Finger,"[254.27, 344.25, 299.03, 356.05, 290.34, 374.2..."
3655,c21528d4223161d8_2,c21528d4223161d8,Tipps,"[300.51, 354.74, 340.36, 364.74, 331.83, 383.2..."
3656,c21528d4223161d8_3,c21528d4223161d8,iPhone,"[506.53, 461.36, 563.77, 477.2, 557.06, 497.19..."


In [ ]:
images = load_images_from_folder('drive/MyDrive/cropped')

In [ ]:
len(images)

2854

In [ ]:
list_of_files = []
for filename in os.listdir('drive/MyDrive/cropped'):
  name = str(filename).strip('.jpg')
  list_of_files.append(name)

In [ ]:
final_image_csvs = final_image_csvs[final_image_csvs['id'].isin(list_of_files)].copy()

In [ ]:
X = np.array(images)
y = final_image_csvs['utf8_string'].to_numpy()

In [ ]:
X.shape

(2854, 32, 128)

In [ ]:
y.shape

(2854,)

In [ ]:
longest_word_length = len(max(y, key=len))

In [ ]:
longest_word_length

31

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100)
X_train, X_vali, y_train, y_vali = train_test_split(X_train, y_train, test_size=0.1, random_state=100)

In [ ]:
char_list = string.ascii_letters+string.digits

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)

    return dig_lst

In [ ]:
inputs = Input(shape=(32,128,1))

conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
batch_norm_5 = BatchNormalization()(conv_5)
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

act_model = Model(inputs, outputs)

act_model.load_weights('crnn_best.hdf5')#trained on 450,000 Mjsynth

In [ ]:
act_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 128, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 32, 128, 64)       640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 64, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 32, 128)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 32, 256)        295168

In [ ]:
labels = Input(name='the_labels', shape=[longest_word_length], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')


def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [ ]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [ ]:
model.load_weights('crnn_best.hdf5')

In [ ]:
train_input_length = np.full(len(X_train), 31)
valid_input_length = np.full(len(X_vali), 31)

train_label_length = []
for x in y_train:
  length = len(x)
  train_label_length.append(length)

valid_label_length = []
for x in y_vali:
  length = len(x)
  valid_label_length.append(length)

In [ ]:
training_img = np.array(X_train)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(X_vali)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

In [ ]:
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)

    return dig_lst

In [ ]:
y_train_encoded = []
for y in y_train:
  y_train_encoded.append(encode_to_labels(y))

y_vali_encoded = []
for y in y_vali:
  y_vali_encoded.append(encode_to_labels(y))

In [ ]:
train_padded_txt = pad_sequences(y_train_encoded, maxlen=longest_word_length, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(y_vali_encoded, maxlen=longest_word_length, padding='post', value = len(char_list))

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
batch_size = 256
epochs = 10
model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


10/10 [==============================] - ETA: 0s - loss: 25.6012
Epoch 1: val_loss improved from inf to 23.52249, saving model to best_model.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 36s 1s/step - loss: 25.6012 - val_loss: 23.5225
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 20.5711
Epoch 2: val_loss improved from 23.52249 to 21.19938, saving model to best_model.hdf5
10/10 [==============================] - 7s 741ms/step - loss: 20.5711 - val_loss: 21.1994
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 18.9105
Epoch 3: val_loss improved from 21.19938 to 19.83033, saving model to best_model.hdf5
10/10 [==============================] - 7s 648ms/step - loss: 18.9105 - val_loss: 19.8303
Epoch 4/10
10/10 [==============================] - ETA: 0s - loss: 18.0566
Epoch 4: val_loss improved from 19.83033 to 18.80186, saving model to best_model.hdf5
10/10 [==============================] - 8s 784ms/step - loss: 18.0566 - val_loss: 18.8019
Epoch 5/10
10/10 [==============================] - ETA: 0s - loss: 17.5160
Epoch 5: val_loss improved from 18.80186 to 18.42097, saving model to best_m

# Testing

In [ ]:
temp = np.array(X_test)

In [ ]:
act_model.load_weights('best_model.hdf5')

In [ ]:
prediction = act_model.predict(temp)

9/9 [==============================] - 3s 198ms/step


In [ ]:
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

# see the results
i = 0
for x in out:
    print("original_text =  ", y_test[i])
    print("predicted text = ", end = '')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    print('\n')
    i+=1

original_text =   www.thebodyshop.co.uk
predicted text = 

original_text =   200
predicted text = 

original_text =   0
predicted text = 

original_text =   and
predicted text = 

original_text =   55
predicted text = 1

original_text =   Town
predicted text = 

original_text =   GRAFFITI
predicted text = S

original_text =   BEER
predicted text = 

original_text =   Options
predicted text = S

original_text =   FREE
predicted text = 

original_text =   97215
predicted text = 

original_text =   -100
predicted text = 1

original_text =   WORLDWIDE
predicted text = M

original_text =   NEED
predicted text = M

original_text =   OUD
predicted text = 

original_text =   07:
predicted text = 

original_text =   1
predicted text = 1

original_text =   to
predicted text = 

original_text =   ICECREAM
predicted text = T

original_text =   5:00
predicted text = 1

original_text =   OZ
predicted text = 1

original_text =   I
predicted text = A

original_text =   S
predicted text = P

original_t

In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.6 MB/s eta 0:00:00


In [ ]:
!pip install python-Levenshtein

In [ ]:
from Levenshtein import distance

act_model.load_weights('crnn_best.hdf5')

#act_model.save('crnn_best.h5', save_format='h5')

distances = []
accuracy = []
accuracy_adjusted = []
correct = []
incorrect = []

prediction = act_model.predict(temp)
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

i = 0
for x in out:
    print("original_text =  ", y_test[i])
    print("predicted text = ", end = '')
    word = ''
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
            word = word + char_list[int(p)]
    print('\n')
    print("WORD: ")
    print(word)
    print()
    distances.append(distance(y_test[i], word))
    tempDist = distance(y_test[i], word)
    print(tempDist)
    if word == y_test[i]:
      print("IDENTICAL")
      accuracy.append(1)
    else:
      print("NOT IDENTICAL")
      accuracy.append(0)
      accuracy_adjusted.append(1/tempDist)
      incorrect.append(word)
      correct.append(y_test[i])
    i+=1

print("DISTANCE: ")
print(np.shape(distances))
print(np.mean(distances))
print("ACCURACY: ")
print(np.shape(accuracy))
print(np.mean(accuracy))
print("ACCURACY ADJUSTED: ")
print(np.shape(accuracy_adjusted))
print(np.mean(accuracy_adjusted))
print("CORRECT SHAPE: ")
print(np.shape(correct))
print("INCORRECT SHAPE: ")
print(np.shape(incorrect))

2/9 [=====>........................] - ETA: 0s

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


9/9 [==============================] - 0s 52ms/step
original_text =   www.thebodyshop.co.uk
predicted text = LE

WORD: 
LE

21
NOT IDENTICAL
original_text =   200
predicted text = CS

WORD: 
CS

3
NOT IDENTICAL
original_text =   0
predicted text = LES

WORD: 
LES

3
NOT IDENTICAL
original_text =   and
predicted text = LLS

WORD: 
LLS

3
NOT IDENTICAL
original_text =   55
predicted text = ALs

WORD: 
ALs

3
NOT IDENTICAL
original_text =   Town
predicted text = WALS

WORD: 
WALS

4
NOT IDENTICAL
original_text =   GRAFFITI
predicted text = 1OOGEH

WORD: 
1OOGEH

8
NOT IDENTICAL
original_text =   BEER
predicted text = JAPANE

WORD: 
JAPANE

6
NOT IDENTICAL
original_text =   Options
predicted text = laels

WORD: 
laels

6
NOT IDENTICAL
original_text =   FREE
predicted text = As

WORD: 
As

4
NOT IDENTICAL
original_text =   97215
predicted text = SS

WORD: 
SS

5
NOT IDENTICAL
original_text =   -100
predicted text = TEANM

WORD: 
TEANM

5
NOT IDENTICAL
original_text =   WORLDWIDE
predicted t

In [56]:
distances_incorrect = []

for i in range(len(correct)):

  print(correct[i].lower())
  print(incorrect[i].lower())
  print()
  distances_incorrect.append(distance(correct[i].lower(), incorrect[i].lower()))

print(np.shape(distances_incorrect))
print('DISTANCES_OF_INCORRECTS: \n' + str(np.mean(distances_incorrect)))

www.thebodyshop.co.uk
le

200
cs

0
les

and
lls

55
als

town
wals

graffiti
1oogeh

beer
japane

options
laels

free
as

97215
ss

-100
teanm

worldwide
aygg

need
hps

oud
cs

07:
als

1
ls

to
ss

icecream
cs

5:00
thls

oz
igt

i
as

s
wee

mm
sls

sleep
mlly

new
ots

fire
sees

forkladd
lls

dun
ls

elix
as

usaf
is

somewhere
cicat

347993
s

much,
als

go
sall

hjalmar
metropoliten

and
as

-james
mbs

cole's
als

nobody)
fs

parking
paee

volume
as

p
abs

9
rors

u
ls

shop
aes

cell
ms

sonntag
ctils

kitchenaid
ss

5
ces

gov.
clage

250
as

ⅺ
as

av
nermni

6
as

s
somang

1
jabs

can
gaes

tea
ms

18
tionne

today
ma

10
linmraur

e63
ss

mta
cree

3
aes

6
cs

forty
as

the
es

the
as

e
as

jeddah
as

human
sms

bu
as

welcome
ss

pan
hlghe

775
yaes

youth
oes

versizon
ws

1896
as

evergreen
ls

12
rushey

zone
neel

bills
ss

for
hs

8
fzt

need
avenue

10
uls

pong
ls

for
at

8
cs

this
polesas

y
expediaca

201
has

3
sls

5
is

by
as

2001
russ

grown
racs

toyo

In [ ]:
annot.to_csv('drive/MyDrive/annot_final.csv')